In [3]:
"""
    Test functionality of chess game with Pytest (and practice TDD)
    Past game data sourced from https://www.kaggle.com/datasets/datasnaek/chess and
    https://github.com/SebLague/Chess-AI/blob/main/Assets/Book/Games.txt
    At best past game data will give me general overview of how well my functions work
    more of statistical confidence than true confidence... other tests needed later...
"""

from constants import *
from board import Board

import pytest
import re
import pandas as pd
game_notes = pd.read_csv('games.csv')
game_notes = game_notes[['victory_status','winner','moves']]

"""
    Requires quick way to initialize board from a mid game position
    Quick way to populate moves for pieces (etc.)
"""

game_notes.head()

,victory_status,winner,moves
0,outoftime,white,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...
1,resign,black,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
2,mate,white,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,mate,white,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,mate,white,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...


In [7]:
game_notes['moves_as_list'] = game_notes['moves'].apply(lambda moves: moves.split())

In [17]:
all_moves = set([move for _, moves in game_notes['moves_as_list'].iteritems() for move in moves])

In [20]:
all_moves

{'Kxa6',
 'Rexb7',
 'R1d7+',
 'N4a5',
 'Kxb4',
 'Rgd4',
 'bxc1=Q',
 'Ned6',
 'a4',
 'Rxc3',
 'Nd6',
 'Qxg1#',
 'Rad4',
 'fxg8=Q+',
 'e5+',
 'Rdxf3',
 'Nxe4',
 'Qxh1',
 'Rcxe5+',
 'Rf7#',
 'Qxf1',
 'Nac4',
 'Rh3',
 'Bb6',
 'Rdg3',
 'Be5+',
 'Rxg1#',
 'R2f3+',
 'Reg3',
 'Nxe8',
 'Rg5',
 'R6xf2',
 'Rag7#',
 'Bc7',
 'Kb2',
 'Rfxd2',
 'bxc3',
 'R1c7+',
 'Nbc4',
 'Qhe1#',
 'Nxd4+',
 'Nfxg8',
 'gxf8=Q',
 'Qec2',
 'Nd7b6',
 'N5b6',
 'Kxb6',
 'Raf8',
 'R8g7+',
 'Kc2+',
 'Qd5+',
 'Qxf5',
 'Rgd2+',
 'R1f5+',
 'Qhf4+',
 'e1=Q',
 'Qeb7+',
 'Qg8+',
 'Rdb6',
 'Qdd6#',
 'R6b7',
 'exd8=Q',
 'd1=Q#',
 'R4e5',
 'Rfg7+',
 'Ne8',
 'Kg6',
 'R1xf7+',
 'Kg4+',
 'Rhd4',
 'Nxc6',
 'Rdc5+',
 'R3e4',
 'Qa4',
 'Ng7',
 'Nc3#',
 'Rea1',
 'Qeb3+',
 'Kf4',
 'Rc5',
 'Rd1d7+',
 'Nxh8',
 'Rxc6#',
 'Bxd3+',
 'Rb8+',
 'Nxg6',
 'Nec5+',
 'Qbc7#',
 'Kxd4',
 'Kc4',
 'Qg3#',
 'Nfxe5',
 'Ngxf7',
 'Rexf7',
 'Rb6#',
 'Nf2',
 'Qf5',
 'R7xa5',
 'Qh1',
 'Rbxa6',
 'Qxf3+',
 'b8=B+',
 'N6xe5',
 'gxh1=Q+',
 'b8=B',
 'Rfh7',
 'Rbe7',
 '

In [46]:
import re

letter_to_piece = {'r':ROOK,'n':KNIGHT,'b':BISHOP,'q':QUEEN,'k':KING}
letter_to_number = {'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,'h':8}
def get_piece_type(move):
    if move[0].isupper():
        return letter_to_piece[move[0].lower()]
    else:
        return PAWN

def get_piece_indicator(move):
    row = None          # returning None signals there is no ambiguity/only one piece of type can do move
    column = None
    lower_case_letters = re.findall('[a-z]',move)  
    if len(lower_case_letters) > 1:
        row = letter_to_number[lower_case_letters[0]] # if there is a column/row indicator, it comes before the to_square
    
    numbers = re.findall('[0-9]',move)
    if len(numbers) > 1:
        column = numbers[0]
        
    return (row, column)

def get_to_square(move):
    row = None
    column = None
    row_column = re.findall('[a-z][0-9]',move)[-1]  # always will be the last lowercase-followed-by-number
    row = letter_to_number[row_column[0]]
    column = row_column[1]
    
    return (row, column)
        

def get_is_take(move):
    if 'x' in move:
        return True
    return False

def get_is_check(move):
    if '+' in move:
        return True
    return False

def get_is_checkmate(move):
    if '#' in move:
        return True
    return False

def get_is_promotion(move):
    if '=' in move:
        return True
    return False

def get_is_king_side_castle(move):
    if 'O-O' == move:
        return True
    return False

def get_is_queen_side_castle(move):
    if 'O-O-O' == move:
        return True
    return False

def parse_move(move):
    piece_type = get_piece_type(move)
    piece_indicator = get_piece_indicator(move)
    to_square = get_to_square(move)
    is_take = get_is_take(move)
    is_check = get_is_check(move)
    is_checkmate = get_is_checkmate(move)
    is_promotion = get_is_promotion(move)
    is_king_side_castle = get_is_king_side_castle(move)
    is_queen_side_castle = get_is_queen_side_castle(move)
    
    return {
            'piece':        piece_type, 
            'piece_indicator': piece_indicator, 
            'to_square':    to_square, 
            'is_take':      is_take, 
            'is_check':     is_check, 
            'is_checkmate': is_checkmate, 
            'is_promotion': is_promotion,
            'is_king_side_castle':is_king_side_castle,
            'is_queen_side_castle':is_queen_side_castle
            }

In [47]:
def turn_to_team(turn):
        if turn % 2:
            return BLACK
        else: 
            return WHITE

char_to_piece = {'p':PAWN,'r':ROOK,'n':KNIGHT,'b':BISHOP,'q':QUEEN,'k':KING}
def fen_to_board(fen):
    board = []
    for char in fen:
        row = []
        
        if char.isnumeric():
            row.append( (EMPTY, EMPTY) for i in range( int(char) )  )
            
        elif char.isalpha():
            
            if char.islower():
                team = BLACK
            else:
                team = WHITE
                
            piece = char_to_piece[char.lower()]

            row.append( ( team, piece ) )

        elif char == '/':
            board.append(row)

        else:
            print('character:', char, 'couldn\'t be translated')
            break
    return board

piece_to_letter = {piece:letter for letter, piece in char_to_piece.items()}
def piece_to_fen(square):
    letter = piece_to_letter[square[1]]
    if square[0] == BLACK:
        return letter.lower()
    elif square[0] == WHITE:
        return letter.upper()
    else: print('Program thinks piece has empty team!!!')

def board_to_fen(board):
    fen = ''
    for row in board:
        empty_count = 0
        for square in row:
            if square == (EMPTY, EMPTY):
                empty_count+=1
            else:
                if empty_count:
                    fen += empty_count
                    empty_count = 0
                fen += piece_to_fen(square)
        fen += '/'
    return fen

standard_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR'
def update_board_from_move(team, move, past_board):
    
    piece_square = find_piece_square(team, board, move)  # requires a lot of game/move logic
    
    piece_info = past_board[piece_square[0]][piece_square[1]]
    
    past_board[piece_square[0]][piece_square[1]] = (EMPTY, EMPTY)
    
    to_square = get_to_square(move)
    past_board[to_square[0]][to_square[1]] = piece_info
    
    return past_board

# ewwwwwwwwwwwwww, but I'm too lazy to change rn
def find_piece_square(team, board, move):
    piece_indication = get_piece_indicator(move)
    to_square = get_to_square(move)
    piece_type = get_piece_type(move)
    
    if not piece_indication[0] and not piece_indication[1]:
        
        matching_pieces = []
        for row_num, row in enumerate(board):
            for column_num, square in enumerate(row):
                if piece_type == square[1] and team == square[0]:
                    matching_pieces.append((row_num, column_num))
        
        if not len(matching_pieces):
            print('No piece matching move')
        if len(matching_pieces) == 1:
            return matching_pieces[0]
        else:
            orientation = -1 if team == WHITE else 1
            
            moves = moveset[piece_type]
            
            for matching_piece in matching_pieces:
                centered_moves = center_moves(moves,matching_piece)
                if piece_type == PAWN and get_is_take(move):
                    centered_moves.append(center_moves([1,1],[1,-1], matching_piece))
                    
                if to_square in centered_moves:
                    return matching_piece
                
                
            
        
    elif piece_indication[0] and piece_indication[1]:
        return piece_indication
    elif piece_indication[0]:
        row = board[piece_indication]
        
        for col, square in enumerate(row):
            if square[1] == piece_type and square[0] == team:
                return (piece_indication[0], col)
            
    elif piece_indication[1]:
        col = piece_indication[0]
        for row_num, row in enumerate(board):
            if row[col][1] == piece_type and row[col][0] == team:
                return (row_num, col)
    



def create_data(df):
    test_data = []
    standard_board_setup = fen_to_board('rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR')
    for i, row in df.iterrows():
        board = standard_board_setup
        
        for turn, move in enumerate(row.moves.split()):
            print(move)
            move_data = parse_move(move)
            move_data['turn'] = turn
            team = turn_to_team(turn)
            move_data['team'] = team
            
            board = update_board_from_move(team, move, board)
                
            move_data['FEN'] = board_to_fen(board)
    return pd.Dataframe(test_data)
            
            

SyntaxError: invalid syntax (<ipython-input-47-7dd7955ae07f>, line 101)

In [45]:
tester = create_data(game_notes.iloc[:5])

d4


NameError: name 'find_piece_square' is not defined